# Map Agent — Gemini URL (no OpenAI)

- Uses Gemini 2.0 Flash via REST (`generativelanguage.googleapis.com`).
- Reads `GEMINI_API_KEY` (and `ORS_API_KEY`) from `part2_implementation/.env`.
- Top-level `await` is used in cells (works in Jupyter).


In [112]:
# Environment: set Gemini model if desired (key is read from .env)
import os
os.environ["GEMINI_MODEL"] = os.environ.get("GEMINI_MODEL", "gemini-2.0-flash")


In [113]:
# Default country filter for OSM geocoding (restrict to Lebanon)
import os
os.environ['OSM_COUNTRYCODES'] = os.environ.get('OSM_COUNTRYCODES', 'lb')


In [115]:
# Import Gemini tool-calling and our servers
import asyncio
from part2_implementation.gemini_provider import run_with_tools
from part2_implementation.agent_sdk_app import TOOLS, AgentsSDKMapAssistant

agent = AgentsSDKMapAssistant()  # used only to dispatch tool calls


**osm Geocoding**

In [116]:
await run_with_tools("geocode tripoli lebanon", TOOLS, agent._dispatch_tool)


{'answer': '',
 'tool_results': [{'tool': 'osm_geocode',
   'content': {'place': 'tripoli lebanon',
    'lat': '34.4373616',
    'lon': '35.8348551',
    'display': 'طرابلس, قضاء طرابلس, محافظة الشمال, 1300, لبنان'}}]}

**Reverse Geocoding (osm)**

In [117]:
await run_with_tools("what is located at lat:34.4373616,lon :35.8348551", TOOLS, agent._dispatch_tool)


{'answer': 'Address: ملك الطاووق, شارع عدنان درويش أغا, Zahrieh, طرابلس, قضاء طرابلس, محافظة الشمال, 1300, لبنان',
 'tool_results': [{'tool': 'osm_reverse',
   'content': {'address': 'ملك الطاووق, شارع عدنان درويش أغا, Zahrieh, طرابلس, قضاء طرابلس, محافظة الشمال, 1300, لبنان'}}]}

**POIs (OSM)**

In [118]:
await run_with_tools("Find restaurant in hamra beirut ", TOOLS, agent._dispatch_tool)


{'answer': 'Top places:\n- Bait al Hamra Guest House, Tawaleet Al Hamra restaurant, شارع بعلبك, الحمرا, الحمراء, رأس بيروت, بيروت, محافظة بيروت, 1103, لبنان (33.8949035, 35.4793113)\n- بربر, شارع بعلبك, الحمرا, الحمراء, رأس بيروت, بيروت, محافظة بيروت, 1103, لبنان (33.8944683, 35.4832320)\n- مطعم أبو نعيم, 12, شارع عمر بن عبد العزيز, الحمرا, الحمراء, رأس بيروت, بيروت, محافظة بيروت, 1103, لبنان (33.8952358, 35.4834994)',
 'tool_results': [{'tool': 'osm_search_poi',
   'content': [{'name': 'Bait al Hamra Guest House, Tawaleet Al Hamra restaurant, شارع بعلبك, الحمرا, الحمراء, رأس بيروت, بيروت, محافظة بيروت, 1103, لبنان',
     'lat': '33.8949035',
     'lon': '35.4793113'},
    {'name': 'بربر, شارع بعلبك, الحمرا, الحمراء, رأس بيروت, بيروت, محافظة بيروت, 1103, لبنان',
     'lat': '33.8944683',
     'lon': '35.4832320'},
    {'name': 'مطعم أبو نعيم, 12, شارع عمر بن عبد العزيز, الحمرا, الحمراء, رأس بيروت, بيروت, محافظة بيروت, 1103, لبنان',
     'lat': '33.8952358',
     'lon': '35.4834994'},
 

## Routing demo (ORS)

In [119]:
await run_with_tools("Find a driving route from Beirut to Tripoli and summarize it.", TOOLS, agent._dispatch_tool)


{'answer': '1. Head east on شارع عبد الحميد الزهراوي\n2. Turn right onto شارع الجندي\n3. Turn right onto جادة إلياس سركيس\n4. Turn right onto شارع البسطة\n5. Keep right\n6. Keep left\n7. Keep left onto جادة الرئيس اللواء فؤاد شهاب, M511\n8. Turn right\n9. Turn right\n10. Keep left onto تقاطع نهر الموت\n11. Keep left\n12. Keep left onto أتستراد البحر, M51\n13. Keep right\n14. Keep right onto تقاطع طرابلس ١\n15. Enter the roundabout and take the 1st exit onto طريق البحر\n16. Continue straight onto A2\n17. Turn left onto طريق البحر, A2\n18. Enter the roundabout and take the 4th exit\n19. Turn right onto شارع البطريرك إلياس الحويك\n20. Keep right onto شارع جبران خليل جبران\n21. Turn right\n22. Turn left onto شارع عدنان درويش أغا\n23. Arrive at شارع عدنان درويش أغا, on the right',
 'tool_results': [{'tool': 'ors_route_places',
   'content': {'distance_km': 82.37,
    'duration_min': 61.9,
    'cumulative_distance_km': 82.37,
    'cumulative_duration_min': 61.9,
    'steps': [{'instruction':

## Inspect server operations

In [120]:
from part2_implementation.servers.osm_server import OSMServer
from part2_implementation.servers.ors_server import ORSServer

[c.name for c in OSMServer().server_params], [c.name for c in ORSServer().server_params]


(['geocode', 'reverse', 'search_poi'], ['route', 'distance', 'nearby'])

## Inspect tool results

In [121]:
res = await run_with_tools("Distance from Beirut to Tripoli?", TOOLS, agent._dispatch_tool)
res['answer'], res['tool_results']


('Distance: 82.37 km',
 [{'tool': 'ors_distance_places',
   'content': {'distance_km': 82.37,
    'origin': [35.5025585, 33.8892265],
    'destination': [35.8348551, 34.4373616]}}])

In [128]:
# Optional: pre-install Gradio to speed first launch
import sys, subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'gradio'])
print('gradio installed')


gradio installed


In [129]:
# Gradio chat UI for the Map Agent
# Requires `gradio`. Installs automatically if missing.
try:
    import gradio as gr
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'gradio'])
    import gradio as gr

def ensure_agent():
    g = globals()
    if 'agent' not in g or 'TOOLS' not in g or 'run_with_tools' not in g:
        from part2_implementation.gemini_provider import run_with_tools as _rwt
        from part2_implementation.agent_sdk_app import TOOLS as _TOOLS, AgentsSDKMapAssistant as _Assistant
        g['run_with_tools'] = _rwt
        g['TOOLS'] = _TOOLS
        g['agent'] = _Assistant()

async def chat_fn(message, history):
    ensure_agent()
    # Small-talk and guidance without hitting the model/tools
    txt = (message or '').strip()
    lower = txt.lower()
    greetings = {'hi','hello','hey','yo','salam','marhaba','hola'}
    map_words = ('route','distance','geocode','reverse','poi','near','nearby','restaurant','restaurants','cafe','cafes','bar','food','hospital','km','lat','lon','coordinate','from','to','driving','walking','cycling')
    if lower in greetings or (len(lower.split()) <= 3 and any(w in lower for w in greetings)):
        return "Hi! I’m your map assistant. Ask for routes, distances, POIs, or geocoding. For example: ‘Distance from Beirut to Tripoli?’ or ‘Find 3 hospitals in Tripoli’."
    if not any(k in lower for k in map_words):
        return "I can help with map tasks: routes, distances, POIs, and geocoding. Try: ‘Route from Beirut to Tripoli’, ‘Geocode Tripoli Lebanon’, or ‘Find 3 hospitals in Tripoli’."

    # Helper: compute haversine distance in km
    from math import radians, sin, cos, asin, sqrt
    def _haversine_km(lat1, lon1, lat2, lon2):
        R = 6371.0
        dlat = radians(lat2 - lat1)
        dlon = radians(lon2 - lon1)
        a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        return round(R * c, 2)

    try:
        res = await run_with_tools(message, TOOLS, agent._dispatch_tool)
    except Exception as e:
        # If the model call fails, fall back to a heuristic single-tool run
        try:
            tool, args = agent._heuristic_route(message)
            tool_res = await agent._dispatch_tool(tool, args)
            if tool in ('ors_distance', 'ors_distance_places') and isinstance(tool_res, dict):
                if 'distance_km' in tool_res:
                    return f"Distance: {tool_res['distance_km']} km"
                if 'origin' in tool_res and 'destination' in tool_res and 'error' in tool_res:
                    try:
                        o_lon, o_lat = tool_res.get('origin')
                        d_lon, d_lat = tool_res.get('destination')
                        return f"Approx distance: {_haversine_km(o_lat, o_lon, d_lat, d_lon)} km (ORS unavailable)"
                    except Exception:
                        pass
            return str(tool_res)[:400]
        except Exception as e2:
            return f'Error: {e} / {e2}'

    # If we got a response, check for ORS errors and retry deterministically
    answer = (res or {}).get('answer') or ''
    tool_results = (res or {}).get('tool_results') or []

    def _has_ors_error(tr_list):
        try:
            for tr in tr_list:
                data = tr.get('content')
                if isinstance(data, dict) and data.get('error'):
                    return data
        except Exception:
            pass
        return None

    err = _has_ors_error(tool_results)
    if ('Distance error:' in answer) or (err and isinstance(err, dict) and 'ORS HTTP' in str(err.get('error',''))):
        try:
            # Force a heuristic tool call preferring *_places
            tool, args = agent._heuristic_route(message)
            if tool == 'ors_distance':
                # Upgrade to places if possible by simple pattern
                import re
                m = re.search(r'from\s+(.+?)\s+to\s+(.+)$', message, flags=re.IGNORECASE) or re.search(r'between\s+(.+?)\s+and\s+(.+)$', message, flags=re.IGNORECASE)
                if m:
                    tool = 'ors_distance_places'
                    args = {'origin_place': m.group(1).strip(), 'destination_place': m.group(2).strip()}
            tool_res = await agent._dispatch_tool(tool, args)
            if isinstance(tool_res, dict) and tool in ('ors_distance', 'ors_distance_places'):
                if 'distance_km' in tool_res:
                    return f"Distance: {tool_res['distance_km']} km"
                if 'error' in tool_res and 'origin' in tool_res and 'destination' in tool_res:
                    try:
                        o_lon, o_lat = tool_res.get('origin')
                        d_lon, d_lat = tool_res.get('destination')
                        return f"Approx distance: {_haversine_km(o_lat, o_lon, d_lat, d_lon)} km (fallback)"
                    except Exception:
                        pass
            # As a last resort, surface the earlier answer
            return answer or 'Sorry, could not compute distance.'
        except Exception:
            return answer or 'Sorry, could not compute distance.'

    return answer

# Build components with backwards compatibility across gradio versions
try:
    chatbot_comp = gr.Chatbot(height=500, type='messages', label='Map Assistant')
    USE_MESSAGES = True
except TypeError:
    try:
        chatbot_comp = gr.Chatbot(height=500, label='Map Assistant')
    except TypeError:
        chatbot_comp = gr.Chatbot(height=500)
    USE_MESSAGES = False
try:
    textbox_comp = gr.Textbox(placeholder='Ask a map question...', lines=2, show_label=False)
except TypeError:
    textbox_comp = gr.Textbox(placeholder='Ask a map question...')

# Explicit Blocks UI with a visible Send button (works across Gradio versions)
_desc = "Ask about routes, distances, POIs, and geocoding. Uses the notebook's map agent and tools."
with gr.Blocks(title='Map Agent (Gradio)') as demo:
    gr.Markdown(_desc)
    # Create components inside Blocks to ensure they render
    try:
        cb = gr.Chatbot(height=500, type='messages', label='Map Assistant')
        use_messages = True
    except TypeError:
        cb = gr.Chatbot(height=500, label='Map Assistant')
        use_messages = False
    with gr.Row():
        try:
            tb = gr.Textbox(placeholder='Ask a map question...', lines=2, show_label=False)
        except TypeError:
            tb = gr.Textbox(placeholder='Ask a map question...')
        send_btn = gr.Button('Send', variant='primary')
    clear_btn = gr.Button('Clear')

    async def _on_send(msg, state):
        try:
            reply = await chat_fn(msg, state)
        except Exception as e:
            reply = f'Error: {e}'
        if use_messages:
            new_state = (state or []) + [
                {"role": "user", "content": msg},
                {"role": "assistant", "content": reply},
            ]
        else:
            new_state = (state or []) + [(msg, reply)]
        return gr.update(value=''), new_state

    def _clear():
        return '', []

    send_btn.click(_on_send, inputs=[tb, cb], outputs=[tb, cb])
    try:
        tb.submit(_on_send, inputs=[tb, cb], outputs=[tb, cb])
    except Exception:
        pass
    clear_btn.click(_clear, outputs=[tb, cb])


demo.launch()


* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
